In [ ]:
# Import required packages
import iris
import tobac
import numpy as np
import pandas as pd
import math
from collections import Counter
from global_land_mask import globe
import os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cmaps
import netCDF4 as nc 
import warnings
import xarray as xr
import seaborn as sns
import logging
#from skimage.morphology import binary_erosion
import multiprocessing
import concurrent
from multiprocessing import Pool
from tqdm import tqdm, notebook
import pickle
import multiprocessing as mp
from itertools import repeat,product
from dask.array import ma, isin
from copy import deepcopy
# Ignore some warnings and append them to the existing filter list
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [ ]:
# Import the 'caffeine' module to prevent the system from going to sleep or the screen from turning off
import caffeine
# Turn on the caffeine mode with the display option set to True
caffeine.on(display=True)

# Load the data for processing.

In [ ]:
year = 2017
# Load the dataframe that contains feature (convective system) information
data = pd.read_hdf(f"/Volumes/Pegasus32 R8/NASA/PROCESSED_TOBAC_NO_NA/{year}/Track.h5",key='/table')

In [ ]:
# Remove cells (convective families) that have only one feature (convective system) in them
data = data.groupby('cell').filter(lambda x : (x['cell'].count()>1)).reset_index(drop=True)

In [ ]:
# Remove redundant columms
data = data.drop(columns=['idx','hdim_1','hdim_2','num','threshold_value'])

In [ ]:
# Rename 'time' column
data.rename(columns = {'time':'datetime'},inplace=True)

In [ ]:
# Convert the 'datetime' column values to a pandas datetime type
data['timestr'] = pd.to_datetime(data['timestr'])

# Split datetime column into separate columns
data['year'] = data['timestr'].dt.year
data['month'] = data['timestr'].dt.month
data['day'] = data['timestr'].dt.day
data['time'] = data['timestr'].dt.time

# Loading pixel level information for every feature (convective system).

In [ ]:
# Load the dictionary from 'tb_dict.pkl' file
with open(f"/Volumes/Pegasus32 R8/NASA/PICKLES_WITH_TB_VALUES_NO_NA/{year}/tb_dict.pkl", 'rb') as handle:
    tb_dict = pickle.load(handle)

# Load the dictionary from 'tau_dict.pkl' file
with open(f"/Volumes/Pegasus32 R8/NASA/PICKLES_WITH_TB_VALUES_NO_NA/{year}/tau_dict.pkl", 'rb') as handle:
    tau_dict = pickle.load(handle)

# Load the dictionary from 'lats_dict.pkl' file
with open(f"/Volumes/Pegasus32 R8/NASA/PICKLES_WITH_TB_VALUES_NO_NA/{year}/lats_dict.pkl", 'rb') as handle:
    lats_dict = pickle.load(handle)

# Load the dictionary from 'lons_dict.pkl' file
with open(f"/Volumes/Pegasus32 R8/NASA/PICKLES_WITH_TB_VALUES_NO_NA/{year}/lons_dict.pkl", 'rb') as handle:
    lons_dict = pickle.load(handle)

In [ ]:
# Replace -1000 with np.nan in dictionary values
for key, value_list in tb_dict.items():
    tb_dict[key] = [np.nan if item == -1000 else item for item in value_list]

In [ ]:
# Replace -1000 with np.nan in dictionary values
for key, value_list in tau_dict.items():
    tau_dict[key] = [np.nan if item == -1000 else item for item in value_list]

In [ ]:
# Replace -1000 with np.nan in dictionary values
for key, value_list in lons_dict.items():
    lons_dict[key] = [np.nan if item == -1000 else item for item in value_list]

In [ ]:
# Replace -1000 with np.nan in dictionary values
for key, value_list in lats_dict.items():
    lats_dict[key] = [np.nan if item == -1000 else item for item in value_list]

In [ ]:
# The map function takes each value from the 'feature' column, 
# looks it up in the dictionary, and returns the corresponding value from the dictionary 
data['TB'] = data.feature.map(tb_dict)
data['TAU'] = data.feature.map(tau_dict)
data['pixel_lons'] = data.feature.map(lons_dict)
data['pixel_lats'] = data.feature.map(lats_dict)

In [ ]:
del tb_dict
del tau_dict
del lons_dict
del lats_dict

In [ ]:
# Columns to replace empty lists with NaN
columns_to_replace = ['TB', 'TAU', 'pixel_lons', 'pixel_lats']

In [ ]:
# Define a function to replace empty lists with NaN
def replace_empty_with_nan(lst):
    return np.nan if isinstance(lst, list) and len(lst) == 0 else lst

In [ ]:
# Apply the replacement function to selected columns
for column in columns_to_replace:
    data[column] = data[column].apply(replace_empty_with_nan)

# Performing aggregation and creating summary statistics of various parameters.

In [ ]:
# Function to calculate length of each list or add nan
def calculate_length_or_nan(lst):
    if isinstance(lst, list):
        length = int(len(lst))
        return length
    else:
        return np.nan

# Apply the function to the column and create a new column
data['pixel_count'] = data['TB'].apply(calculate_length_or_nan)

In [ ]:
# Define a threshold
threshold = 220

# Custom function to count values below a certain threshold
def count_below_threshold(lst):
    if isinstance(lst, list):
        return sum(1 for value in lst if value <= threshold)
    else:
        return np.nan
    
# Create a new column with the count of values below the threshold
data['pixels_below_220'] = data['TB'].apply(count_below_threshold)

In [ ]:
# Define a threshold
threshold = 200

# Custom function to count values below a certain threshold
def count_below_threshold(lst):
    if isinstance(lst, list):
        return sum(1 for value in lst if value <= threshold)
    else:
        return np.nan

# Create a new column with the count of values below the threshold
data['pixels_below_200'] = data['TB'].apply(count_below_threshold)

In [ ]:
# Calculation convective fraction
data['convective_fraction'] = data['pixels_below_220']/data['pixel_count']*100

In [ ]:
# Sort values by 'cell' number (convective family) and time step of each convective system within convective family
data = data.sort_values(by=['cell','time_cell']).reset_index(drop=True)

In [ ]:
# Find minimum brightnesss temperature of each CS (convectvie system)
data['minTB_feature'] = data['TB'].apply(np.nanmin)

In [ ]:
# Find minimum brightnesss temperature of each convective family
dfc = data.groupby('cell')['minTB_feature']
data['minTB_cell' ] = dfc.transform('min')

In [ ]:
# Find average brightnesss temperature of each CS (convectvie system)
data['avgTB_feature'] = data['TB'].apply(np.nanmean)

In [ ]:
# Find average brightnesss temperature of each convective family
dfc = data.groupby('cell')['avgTB_feature']
data['avgTB_cell' ] = dfc.transform('mean')

In [ ]:
# Find maximum brightnesss temperature of each CS (convectvie system)
data['maxTB_feature'] = data['TB'].apply(np.nanmax)

In [ ]:
# Find maximum brightnesss temperature of each convective family
dfc = data.groupby('cell')['maxTB_feature']
data['maxTB_cell' ] = dfc.transform('max')

# Calculate percentiles

In [ ]:
tb = data['TB']

In [ ]:
tenth = [np.nanpercentile(i, 10) for i in tqdm(tb)]

In [ ]:
data['10th'] = tenth

In [ ]:
twentyfifth = [np.nanpercentile(i, 25) for i in tqdm(tb)]

In [ ]:
data['25th'] = twentyfifth

In [ ]:
fifty = [np.nanpercentile(i, 50) for i in tqdm(tb)]

In [ ]:
data['50th'] = fifty

In [ ]:
seventyfive = [np.nanpercentile(i, 75) for i in tqdm(tb)]

In [ ]:
data['75th'] = seventyfive

In [ ]:
ninety = [np.nanpercentile(i, 90) for i in tqdm(tb)]

In [ ]:
data['90th'] = ninety

In [ ]:
ninetyfive = [np.nanpercentile(i, 95) for i in tqdm(tb)]

In [ ]:
data['95th'] = ninetyfive

In [ ]:
ninetynine = [np.nanpercentile(i, 99) for i in tqdm(tb)]

In [ ]:
data['99th'] = ninetynine

# Continuing calculation of other parameters

In [ ]:
# Calculate standard deviation of brightness temperature
# Create a new column with the standard deviation of each list
data['std_dev_tb'] = data['TB'].apply(np.nanstd)

In [ ]:
radius_earth = 6371  # Earth's radius in kilometers
degree_to_radian = math.pi / 180.0
constant = 0.1*0.1*(2 * math.pi * radius_earth / 360)*(2 * math.pi * radius_earth / 360)
def calculate_radius(lat_degrees, pixel_count):
    # Convert latitude from degrees to radians
    lat_radians = lat_degrees * degree_to_radian
    # Calculate the area
    area = math.cos(lat_radians)*constant*pixel_count
    radius = np.sqrt(area/math.pi)
    return radius

In [ ]:
# Apply the function to create the 'radius2' column
data['radius'] = data.apply(lambda row: calculate_radius(row['latitude'], row['pixel_count']), axis=1)

In [ ]:
# Find maximum radius of each convective family
dfc = data.groupby('cell')['radius']
data['max_radius_cell' ] = dfc.transform('max')

In [ ]:
# Calculate total hours and add as a new column
data['total_hours'] = data['time_cell'].apply(lambda td: td.days * 24 + td.seconds // 3600)

In [ ]:
# Calculate maximum duration of each convective family
dfc = data.groupby('cell')['total_hours']
data['lifetime_hours' ] = dfc.transform('max')

In [ ]:
# Calculate duration/lifetime of each convective family as number of convective system it contains.
dfc = data.groupby('cell')['feature']
data['lifetime_num_cs' ] = dfc.transform('count')

In [ ]:
# Calculate average optical thickness of CS (convective system)
data['avg_optical_thickness'] = data['TAU'].apply(np.nanmean)
# Calculate maximum optical thickness of CS (convective system)
data['max_optical_thickness'] = data['TAU'].apply(np.nanmax)

In [ ]:
# Squared correlation of latitude and longitude within convective system
data['squared_corr'] = data.apply(lambda row: (np.corrcoef(row['pixel_lats'],row['pixel_lons'])[0,1])**2,axis=1)

# Calculating wind speed and wind direction

In [ ]:
def wind_speed(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    # Radius of the Earth in kilometers
    earth_radius = 6371.0
    # Haversine formula
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = earth_radius * c
    speed_kmh = distance/3
    speed_ms = speed_kmh * (5/18)
    return speed_ms

In [ ]:
# Calculate wind speed for each cell's (convective family's) consecutive latitude and longitude pairs
def calculate_wind_speed(group):
    speeds = []
    total = len(group) - 1
    for i in range(total):
        lat1, lon1 = group.iloc[i]['latitude'], group.iloc[i]['longitude']
        lat2, lon2 = group.iloc[i + 1]['latitude'], group.iloc[i + 1]['longitude']
        speeds.append(wind_speed(lat1, lon1, lat2, lon2))
    speeds.append(np.nan)  # Adding np.nan for the last row
    return speeds

# Apply the calculate_wind_speed function to each cell group
result = data.groupby('cell').apply(calculate_wind_speed)

In [ ]:
# Update the wind speed values in the existing dataframe based on wind_speed_series
for cell, wind_speed_list in tqdm(result.items()):
    row_indices = data[data['cell'] == cell].index
    for i, wind_speed in enumerate(wind_speed_list):
        data.at[row_indices[i], 'wind_speed'] = wind_speed

In [ ]:
# Function to convert wind directions from degrees to letters
def wind_direction_to_letter(degrees):
    if pd.notna(degrees):  # Check if the value is not NaN
        directions = ["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]
        index = int((degrees + 11.25) % 360 / 22.5)
        return directions[index]
    else:
        return np.nan 

In [ ]:
# Create empty list to append wind direction values to
wind_directions = []
# Iterate over rows to calculate wind direction of consecutive convection systems
for i,row in data.iterrows():
    try:
        u_wind = data.iloc[i]['longitude'] - data.iloc[i+1]['longitude']
        v_wind = data.iloc[i]['latitude'] - data.iloc[i+1]['latitude']
        angle_radians = math.atan2(u_wind, v_wind)
        angle_degrees = math.degrees(angle_radians)
        angle_degrees = angle_degrees % 360# Convert to direction with respect to north
        wind_directions.append(angle_degrees)
    except:
        pass
wind_directions.append(np.nan) #instead of the last row

In [ ]:
# Add values from the list as a dataframe column
data['wind_dir'] = wind_directions

In [ ]:
# Define a function to replace last values with NaN
def replace_last_with_nan(column):
    column.iloc[-1] = np.nan
    return column

# Group by 'cell' column and apply the above function
data['wind_dir'] = data.groupby('cell')['wind_dir'].apply(replace_last_with_nan)

In [ ]:
# Apply function to convert wind directions from degress to letters
data['wind_dir_letter'] = data['wind_dir'].apply(wind_direction_to_letter)

# Calculation of miscellaneous parameters

In [ ]:
# Function to calculate the minimum value from a list
def calculate_min(lst):
    return min(lst) if isinstance(lst, list) else None
# Function to calculate the maximum value from a list
def calculate_max(lst):
    return max(lst) if isinstance(lst, list) else None
# Apply the function to determine minimum and maximum coordinates of each convective system
data['min_lat'] = data['pixel_lats'].apply(calculate_min)
data['max_lat'] = data['pixel_lats'].apply(calculate_max)
data['min_lon'] = data['pixel_lons'].apply(calculate_min)
data['max_lon'] = data['pixel_lons'].apply(calculate_max)

In [ ]:
latitudes = list(data['pixel_lats'])
longitudes = list(data['pixel_lons'])
# Function to process every convective system and determine land/water mask
# More infromation about global_land_mask module that was used to determine
# whether a datapoint is located over land or water can be found here:
# https://github.com/toddkarin/global-land-mask
def process_row(latitude_list, longitude_list):
    if isinstance(latitude_list, list) and isinstance(longitude_list, list):
        longitude_list = [l - 180 for l in longitude_list]
        lat_lon = [globe.is_land(la, lo) for la, lo in zip(latitude_list, longitude_list)]
        outcome_counts = Counter(lat_lon)
        majority_outcome = max(outcome_counts, key=outcome_counts.get)
        return majority_outcome
    else:
        return np.nan

In [ ]:
# Splitting data into chucks for smoother processing
chunk_size = 10000
chunks_lats = [latitudes[i:i + chunk_size] for i in range(0, len(latitudes), chunk_size)]
chunks_lons = [longitudes[i:i + chunk_size] for i in range(0, len(longitudes), chunk_size)]

In [ ]:
# Using parallel processing and appending results to 'land_water_results' list
land_water_results = []
for lats, lons in zip(chunks_lats, chunks_lons):
    with concurrent.futures.ProcessPoolExecutor(mp_context=mp.get_context('fork'), max_workers=28) as executor:
        results = list(notebook.tqdm(executor.map(process_row, lats, lons), total=len(lats)))
        land_water_results.append(results)

In [ ]:
# Concatenate a list of lists
merged_list = sum(land_water_results, [])

In [ ]:
# Add the new list as a new column
data['land_water_mask'] = merged_list

In [ ]:
# Replace True and False values with land and water values respectively
data['land_water_mask'] = data['land_water_mask'].replace({True: 'land', False: 'water'})

In [ ]:
# Defining function to calculate overlap between convective system and using parallel processing to process the data
def calculate_overlap(index):
    try:
        if isinstance(data.iloc[index]['pixel_lons'], list) and isinstance(data.iloc[index + 1]['pixel_lons'], list):
            list1 = [*zip(data.iloc[index]['pixel_lons'], data.iloc[index]['pixel_lats'])]
            list2 = [*zip(data.iloc[index + 1]['pixel_lons'], data.iloc[index + 1]['pixel_lats'])]
            coinciding_count = sum(tuple1 in list2 for tuple1 in list1)
            biggest_cs = max(len(data.iloc[index]['pixel_lons']), len(data.iloc[index + 1]['pixel_lons']))
            percent_overlap = coinciding_count / biggest_cs * 100
            return percent_overlap
        else:
            return np.nan
    except:
        pass

chunk_size = 10000
inds = data.index.values
chunk_inds = [inds[i:i + chunk_size] for i in range(0, len(inds), chunk_size)]

percent_overlap = []
for ind in chunk_inds:
    with concurrent.futures.ProcessPoolExecutor(mp_context=mp.get_context('fork'), max_workers=28) as executor:
        results = list(notebook.tqdm(executor.map(calculate_overlap, ind), total=len(ind)))
        percent_overlap.append(results)

In [ ]:
data

In [ ]:
# Concatenate a list of lists
merged_list = sum(percent_overlap, [])

In [ ]:
# Add the new list as a new column
data['percent_overlap'] = merged_list

In [ ]:
# Define a function to replace last convective system of each convective family with NaN
def replace_last_with_nan(column):
    column.iloc[-1] = np.nan
    return column

# Group by 'cell' column and apply the custom function
data['percent_overlap'] = data.groupby('cell')['percent_overlap'].apply(replace_last_with_nan)

In [ ]:
# Calculate percent non overlap between consecutive convective systems
data['percent_non_overlap'] = 100 - data['percent_overlap']

In [ ]:
data

In [ ]:
data2 = data.dropna(subset=['pixel_lats', 'pixel_lons', 'TB'])

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the Haversine distance between two points on the Earth's surface
    specified in decimal degrees of latitude and longitude.

    :param lat1: Latitude of the first point
    :param lon1: Longitude of the first point
    :param lat2: Latitude of the second point
    :param lon2: Longitude of the second point
    :return: Haversine distance in kilometers
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    radius_of_earth = 6371  # Earth's radius in kilometers
    distance = radius_of_earth * c

    return distance

In [ ]:
# The spatial gradient of convective systems
# Determined by the maximum and minimum temperature and their separation distance, K/km)
def calculate_gradient(row):
    diff = abs(min(row['TB']) - max(row['TB']))
    distance = haversine_distance(row['min_lat'], row['min_lon'], row['max_lat'], row['max_lon'])
    if distance == 0:
        return np.nan
    
    res = (diff / distance) * 1000
    return res

In [ ]:
# Use tqdm with progress_apply to apply the function and display a progress bar
tqdm.pandas(desc="Processing")
data2['cs_gradient'] = data2.progress_apply(calculate_gradient, axis=1)

In [ ]:
# Create a dictionary from the DataFrame
gradient_dict = dict(zip(data2['feature'], data2['cs_gradient']))

In [ ]:
data['cs_gradient'] = data.feature.map(gradient_dict)

# Calculate ellipse parameters

In [ ]:
data

In [ ]:
# Find central latitude of each CS (convectvie system)
data['central_latitude'] = data['pixel_lats'].apply(np.nanmean)

In [ ]:
# Find central longitude of each CS (convectvie system)
data['central_longitude'] = data['pixel_lons'].apply(np.nanmean)

In [ ]:
def calculate_major_minor_params(lats,lons):
     # Check for empty lists or NaN values
    if not isinstance(lats, list) or not isinstance(lons, list) or len(lats) == 1 or len(lons) == 1:
        return np.nan, np.nan,np.nan
    
    # Combine latitude and longitude into a single NumPy array
    points = np.array(list(zip(lats, lons)))

    # Perform Principal Component Analysis (PCA)
    # Reshape points array to ensure it's 2D
    points = points.reshape((-1, 2))

    covariance_matrix = np.cov(points, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

    # Sort eigenvalues and eigenvectors in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Calculate semi-major and semi-minor axes
    semi_major = 2 * np.sqrt(eigenvalues[0])  # Largest eigenvalue
    semi_minor = 2 * np.sqrt(eigenvalues[1])  # Second largest eigenvalue
    # Calculate inclination of the major axis
    inclination = np.degrees(np.arctan2(eigenvectors[1, 1], eigenvectors[0, 1]))

    return semi_major, semi_minor,inclination

In [ ]:
latitudes = data['pixel_lats']
longitudes = data['pixel_lons']

In [ ]:
results = []
for a,b in tqdm(zip(latitudes,longitudes)):
    r = calculate_major_minor_params(a,b)
    results.append(r)

In [ ]:
# Create a DataFrame with columns 'semi_major', 'semi_minor', 'inclination'
new_df = pd.DataFrame(results, columns=['semi_major', 'semi_minor', 'inclination'])

In [ ]:
data = pd.concat([data, new_df], axis=1)

In [ ]:
data['eccentricity'] = np.sqrt(np.square(data['semi_major']) - np.square(data['semi_minor'])) / data['semi_major']

# Saving the data

In [ ]:
# Remove redundant columns
data = data.drop(columns=['timestr','TB','TAU','pixel_lats','pixel_lons'])

In [ ]:
# Assuming 'data' is your DataFrame and 'time' column contains datetime.time objects
data['time'] = data['time'].apply(lambda x: x.strftime('%H:%M:%S'))

In [ ]:
data.to_csv(f'{year}.csv',index=False)

In [ ]:
ds = xr.Dataset.from_dataframe(data)

In [ ]:
netcdf_file_path = f'{year}.nc'

In [ ]:
ds.to_netcdf(netcdf_file_path)